In [35]:

# import sys
# import os

# # # Ensure the project root is correctly in sys.path
# # project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
# # if project_root not in sys.path:
#     sys.path.insert(0, project_root)

# # # Correct data paths
# # artists_path = os.path.join(project_root, "data", "artists.csv")
# # conflicts_path = os.path.join(project_root, "data", "conflicts.csv")


In [36]:
import sys

sys.path.append("..")

In [37]:
from musical_festival_lineup.musical_festival_lineup import MusicalFestivalSolution
from musical_festival_lineup.musical_festival_data import MusicalFestivalData
from musical_festival_lineup.utils import visualize_musical_lineup
import numpy as np
import time

In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. Structuring the problem

The  objective  is  to  design  the  optimal  festival  lineup  by  scheduling  artists  across  stages  and 
time  slots while:
- Maximizing  prime  slot  popularity
- Ensuring  genre  diversity  among  stages 
- Minimizing  fan conflicts  at each time slot. 

__`Details`__

- **Artists**: An  artist is  characterized  by  a  popularity  score  (from  0  to  100) a genre  and  fan  base.
    <br>This information with be on __`artists`__ from musical_festival_data

- **Conflicts**:   with  other  artists  (score  from  0  to  1).  Conflicts  arise  when  artists  with  overlapping 
fan  bases  perform  at  the  same  time  but  on  different  stages,  which  can  negatively  impact 
attendance.
    <br>This information will be on __`conflits`__ from musical_festival_data


### 2. Defining a  solution/individual

Each  individual  represents  a  festival  lineup,  specifying  which  artist  plays  on  which  stage  and 
at  which  time  slot. 
These  are  the  constraints  : 
- Each artist is assigned to exactly one stage and slot 
- All time slots start and end at the same time. 
- All stages have the same amount of slots
- These artists should be distributed across  **5 stages  with  7 time slots each**

__`Important`__ 
<br>Any  lineup  where  an  artist  is  assigned  to  **multiple  time  slots  or 
left  unassigned** is  not  part  of  the  search  space  and  is  not  considered  a  solution.  It  is 
forbidden to generate such an arrangement during evolution. 


__`we propose a  having  a solution as`__:

- A string of 5 (stages)*7(slots)= 35 numbers of **1** to **35**
- As for the genetic algorithms, we should have string of the same size, we will be appending a 0 to 1 digit numbers.
- So the solution will be a string of 35 * 2 (two character)= so will have i string of 70

Here a individual of our problem:

In [39]:
data = MusicalFestivalData()

In [40]:
lineup=MusicalFestivalSolution(data=data, repr="2406083423021829103033012519173112162614150532070320000422092728131121")
",".join([ str(i) for i in lineup.repr])

'24,6,8,34,23,2,18,29,10,30,33,1,25,19,17,31,12,16,26,14,15,5,32,7,3,20,0,4,22,9,27,28,13,11,21'

In [41]:
# Python code to show a random initialized solution
lineup=MusicalFestivalSolution(data=data)
",".join([ str(i) for i in lineup.repr])

'34,2,1,23,6,27,33,15,18,32,14,10,16,13,29,19,5,0,3,25,26,7,31,11,12,20,24,8,17,30,22,9,28,21,4'

- Having that, we have to options:
- 1. The first 7 numbers represent the stage 1 , having each of the 7 representing the slots in  order (1 to 7) by the slot?
- 2. Or the first 5 would represent the slot 1 (first time slot) , having the each of the 5 representing the stages in order (1 to 5)?

For that we choose the option 2, because how our fitness will be defined. It is well explained in the session 3, when defining fitness


In [42]:
visualize_musical_lineup("2406083423021829103033012519173112162614150532070320000422092728131121", data.artists)

,Slot,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
0,Slot 1,24: Shadow Cadence|Jazz|66,6: Aurora Skies|Pop|75,8: Crimson Harmony|Classical|20,34: Parallel Dimension|Electronic|58,23: Electric Serpents|Electronic|99
1,Slot 2,2: Velvet Pulse|Jazz|35,18: Velvet Underground|Rock|72,29: Harmonic Dissonance|Classical|96,10: The Wandering Notes|Jazz|84,30: Turbo Vortex|Rock|53
2,Slot 3,33: Cosmic Frequency|Rock|53,1: Solar Flare|Electronic|78,25: Rhythm Alchemy|Jazz|94,19: Astral Tide|Electronic|69,17: Nightfall Sonata|Classical|84
3,Slot 4,31: The Jazz Nomads|Jazz|64,12: Blue Horizon|Pop|51,16: Mystic Rhythms|Classical|78,26: Cloud Nine Collective|Pop|97,14: Synthwave Saints|Rock|94
4,Slot 5,15: Golden Ember|Rock|61,5: Echo Chamber|Electronic|98,32: The Bassline Architects|Hip-Hop|61,7: Static Mirage|Rock|94,3: Neon Reverie|Electronic|100
5,Slot 6,20: The Sonic Drifters|Rock|88,0: Midnight Echo|Rock|75,4: The Silver Owls|Classical|85,22: Quantum Beat|Hip-Hop|96,9: Deep Resonance|Jazz|90
6,Slot 7,27: Hypnotic Echoes|Rock|77,28: The Polyrhythm Syndicate|Jazz|66,13: Lunar Spectrum|Rock|99,11: Phantom Groove|Hip-Hop|47,21: Celestial Voyage|Electronic|95


In [43]:
visualize_musical_lineup(lineup.repr, data.artists)

,Slot,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
0,Slot 1,34: Parallel Dimension|Electronic|58,2: Velvet Pulse|Jazz|35,1: Solar Flare|Electronic|78,23: Electric Serpents|Electronic|99,6: Aurora Skies|Pop|75
1,Slot 2,27: Hypnotic Echoes|Rock|77,33: Cosmic Frequency|Rock|53,15: Golden Ember|Rock|61,18: Velvet Underground|Rock|72,32: The Bassline Architects|Hip-Hop|61
2,Slot 3,14: Synthwave Saints|Rock|94,10: The Wandering Notes|Jazz|84,16: Mystic Rhythms|Classical|78,13: Lunar Spectrum|Rock|99,29: Harmonic Dissonance|Classical|96
3,Slot 4,19: Astral Tide|Electronic|69,5: Echo Chamber|Electronic|98,0: Midnight Echo|Rock|75,3: Neon Reverie|Electronic|100,25: Rhythm Alchemy|Jazz|94
4,Slot 5,26: Cloud Nine Collective|Pop|97,7: Static Mirage|Rock|94,31: The Jazz Nomads|Jazz|64,11: Phantom Groove|Hip-Hop|47,12: Blue Horizon|Pop|51
5,Slot 6,20: The Sonic Drifters|Rock|88,24: Shadow Cadence|Jazz|66,8: Crimson Harmony|Classical|20,17: Nightfall Sonata|Classical|84,30: Turbo Vortex|Rock|53
6,Slot 7,22: Quantum Beat|Hip-Hop|96,9: Deep Resonance|Jazz|90,28: The Polyrhythm Syndicate|Jazz|66,21: Celestial Voyage|Electronic|95,4: The Silver Owls|Classical|85


- Get artists of slot

In [44]:
lineup._get_slot_repr_list(slot=6)

[22, 9, 28, 21, 4]

### 3. Defining fitness

" The  quality  of  a  festival  lineup  is  determined  by  balancing  __`three  equally`__  important 
objectives** ,  each  contributing  to  an  overall  score.  Since  these  objectives  have  different 
scales,  ** they  must  be  normalized  to  the  same  range  (between  0  and  1)  to  ensure  they 
contribute equally to the final fitness score.
<br>
 These are the objectives: 
 
__`Prime  Slot  Popularity`__  :  The  most  popular  artists  should  be  scheduled  in  the  prime 
slots  (**the  last  time  slot  on  each  stage**).  This  score  is  calculated  by  normalizing  the 
total  popularity  of  artists  in  prime  slots  against  the  maximum  possible  total  popularity 
(e.g. if only most popular artists - score 100 - were scheduled on the prime slot). 

__`Genre  Diversity`__  :  A  diverse  distribution  of  **genres  among  stages  in  each  time  slot**
improves  the  festival  experience.  This  score  is  determined  by  normalizing  the 
number  of  unique  genres  in  each  slot  relative  to  the  maximum  possible  number  of 
unique  genres  (e.g.  if  only  different  genres  were  scheduled  in  that  time  slot)  .  Then 
you take the average among time slots. 

__`Conflict  Penalty`__  : ** Fan  conflicts  occur  when  artists  with  overlapping  audiences 
perform  at  the  same  time  on  different  stages**. 
 This  score  is  calculated  by  normalizing 
the  total  conflict  value  in  each  slot  against  the  worst  possible  conflict  scenario  (e.g. 
where  all  artists  with  maximum  conflict  are  scheduled  together).  Then  you  take  the 
average  among  time  slots.  Since  conflicts  negatively  impact  the  lineup,  this  is  a 
penalization  score. 
"

__`Prime  Slot  Popularity`__  

1.  Maximum  possible  total  popularity, having into account that There is 5 stages, so we eill be summing the 5(NUM STAGES) largest

In [45]:
data.max_popularity_in_prime_slot

493

2. Total  popularity  of  artists: <br>
For that, it was defined a function that receives artists Ids and returns the popularity for that specifics Ids. <br>
Lets test it!



In [46]:
#Summing popularity of all artists
data.get_sum_popularity(list(range(14)))

1031

In [47]:
#Summing popularity of [1,2,3,4]
data.get_sum_popularity([1,2,3,4])

298

3. Normalizing the popularity against the max number of popularity

In [48]:
lineup._get_popularity_normalized([1,2,3,4])

0.6044624746450304

__`Genre  Diversity`__  : 

1. Get maximum number of genres per slot, can not be greater than the total number of stages, which is 5

In [49]:
#Genre  Diversity
data.max_distinct_genre_per_slot

5

2. Number of distinct genres : <br>
For that, it was defined a function that receives artists Ids and returns distinct genres in that list of specifics Ids. <br>
Lets test it!


In [50]:
data.get_count_distinct_genres(artists_ids_list=[0,1,10,2])

3

3. Normalizing the genres against the max number of distinct genres

In [51]:
lineup._get_genre_diversity_normalized(artists_ids_list=[0,1,10])

0.6

__`Penalty Conflict`__  : 

1. Get maximum possible conflift, the  worst  possible  conflict  scenario, so for that we get the top K worst case scenerio. <br>
 What is K?
 Is the numbers os conflits in a slot, which is the combination of 5 elements when grouping in two so, we will have C5,2, tthat is equal to 10.
 So will will be choosing the top 10 worst conflits


In [52]:
data.max_worst_conflit_per_slot

np.float64(10.0)

2. The  Total  conflict  value  in  each  slot : <br>
For that, it was defined a function that receives artists Ids and sum of all conflicts in the list. <br>
Lets test it!


In [53]:
data.get_sum_conflicts(artists_ids_list=[3,1,10])

np.float64(1.8)

In [54]:
data.get_sum_conflicts(artists_ids_list=[32, 29, 25, 13, 30])

np.float64(5.75)

In [55]:
data.get_sum_conflicts(artists_ids_list=[2,10])

np.float64(0.9)

3. Normalizing the conflicts against the worst conflit scenario

In [56]:
lineup._get_conflicts_normalized(artists_ids_list=[3,1,10])

np.float64(0.18)

In [57]:
lineup._get_conflicts_normalized(artists_ids_list=[32, 29, 25, 13, 30])

np.float64(0.575)

##### 3.2 Other important functions in the solution

In [58]:
lineup.repr

[34,
 2,
 1,
 23,
 6,
 27,
 33,
 15,
 18,
 32,
 14,
 10,
 16,
 13,
 29,
 19,
 5,
 0,
 3,
 25,
 26,
 7,
 31,
 11,
 12,
 20,
 24,
 8,
 17,
 30,
 22,
 9,
 28,
 21,
 4]

- Get the list of artists Ids in a Slot, Slot is a parameter

In [59]:
lineup._get_slot_repr_list(1)

[27, 33, 15, 18, 32]

In [60]:
lineup._get_slot_repr_list(0)

[34, 2, 1, 23, 6]

##### 3.3 Fitness

In [61]:
#Random 1
lineup=MusicalFestivalSolution(data=data)
lineup.repr
lineup.fitness(verbose=True)

Slot of artists: [17, 1, 26, 16, 10]
Slot 0: Conflitcs: 0.39499999999999996, genres: 0.8, sum_popularity: 0
Slot 0: List of  Conflitcs: [np.float64(0.39499999999999996)], List of genres: [0.8], Popularity of the prime slot: 0
Slot of artists: [24, 13, 3, 2, 6]
Slot 1: Conflitcs: 0.4950000000000001, genres: 0.8, sum_popularity: 0
Slot 1: List of  Conflitcs: [np.float64(0.39499999999999996), np.float64(0.4950000000000001)], List of genres: [0.8, 0.8], Popularity of the prime slot: 0
Slot of artists: [29, 30, 28, 21, 11]
Slot 2: Conflitcs: 0.35, genres: 1.0, sum_popularity: 0
Slot 2: List of  Conflitcs: [np.float64(0.39499999999999996), np.float64(0.4950000000000001), np.float64(0.35)], List of genres: [0.8, 0.8, 1.0], Popularity of the prime slot: 0
Slot of artists: [5, 15, 18, 12, 0]
Slot 3: Conflitcs: 0.54, genres: 0.6, sum_popularity: 0
Slot 3: List of  Conflitcs: [np.float64(0.39499999999999996), np.float64(0.4950000000000001), np.float64(0.35), np.float64(0.54)], List of genres: [0.

np.float64(1.1056128658359898)

In [62]:
#Random 2
lineup=MusicalFestivalSolution(data=data)
lineup.repr
lineup.fitness(verbose=True)

Slot of artists: [4, 27, 11, 29, 13]
Slot 0: Conflitcs: 0.32999999999999996, genres: 0.6, sum_popularity: 0
Slot 0: List of  Conflitcs: [np.float64(0.32999999999999996)], List of genres: [0.6], Popularity of the prime slot: 0
Slot of artists: [16, 31, 14, 30, 9]
Slot 1: Conflitcs: 0.465, genres: 0.6, sum_popularity: 0
Slot 1: List of  Conflitcs: [np.float64(0.32999999999999996), np.float64(0.465)], List of genres: [0.6, 0.6], Popularity of the prime slot: 0
Slot of artists: [5, 7, 25, 19, 23]
Slot 2: Conflitcs: 0.6450000000000001, genres: 0.6, sum_popularity: 0
Slot 2: List of  Conflitcs: [np.float64(0.32999999999999996), np.float64(0.465), np.float64(0.6450000000000001)], List of genres: [0.6, 0.6, 0.6], Popularity of the prime slot: 0
Slot of artists: [22, 18, 1, 3, 32]
Slot 3: Conflitcs: 0.43, genres: 0.6, sum_popularity: 0
Slot 3: List of  Conflitcs: [np.float64(0.32999999999999996), np.float64(0.465), np.float64(0.6450000000000001), np.float64(0.43)], List of genres: [0.6, 0.6, 0.

np.float64(0.862563025210084)

In [63]:
#Random 3
lineup=MusicalFestivalSolution(data=data)
lineup.repr
lineup.fitness(verbose=True)

Slot of artists: [12, 17, 29, 21, 9]
Slot 0: Conflitcs: 0.395, genres: 0.8, sum_popularity: 0
Slot 0: List of  Conflitcs: [np.float64(0.395)], List of genres: [0.8], Popularity of the prime slot: 0
Slot of artists: [16, 19, 15, 30, 32]
Slot 1: Conflitcs: 0.295, genres: 0.8, sum_popularity: 0
Slot 1: List of  Conflitcs: [np.float64(0.395), np.float64(0.295)], List of genres: [0.8, 0.8], Popularity of the prime slot: 0
Slot of artists: [28, 11, 10, 24, 18]
Slot 2: Conflitcs: 0.515, genres: 0.6, sum_popularity: 0
Slot 2: List of  Conflitcs: [np.float64(0.395), np.float64(0.295), np.float64(0.515)], List of genres: [0.8, 0.8, 0.6], Popularity of the prime slot: 0
Slot of artists: [2, 20, 26, 14, 34]
Slot 3: Conflitcs: 0.615, genres: 0.8, sum_popularity: 0
Slot 3: List of  Conflitcs: [np.float64(0.395), np.float64(0.295), np.float64(0.515), np.float64(0.615)], List of genres: [0.8, 0.8, 0.6, 0.8], Popularity of the prime slot: 0
Slot of artists: [6, 27, 22, 31, 25]
Slot 4: Conflitcs: 0.485,

np.float64(0.9802506519849319)

### Hill-Climbing

In [64]:
from library.algorithms.hill_climbing import hill_climbing
from musical_festival_lineup.musical_festival_lineup_hill_climbing import MusicalFestivalHillClimbingSolution

In [65]:
lineup=MusicalFestivalHillClimbingSolution(data=data)

In [66]:
solution = hill_climbing(lineup, maximization=True, max_iter=1000, verbose=True)
print(solution.fitness())

Current solution: [8, 22, 2, 0, 6, 10, 13, 11, 16, 17, 26, 25, 20, 15, 21, 12, 30, 27, 1, 7, 33, 34, 9, 3, 23, 28, 14, 18, 19, 31, 32, 4, 24, 5, 29] with fitness 1.0249579831932771
Neighbor: [8, 22, 2, 0, 6, 10, 13, 11, 16, 17, 26, 25, 20, 15, 21, 23, 30, 27, 1, 7, 33, 34, 9, 3, 12, 28, 14, 18, 19, 31, 32, 4, 24, 5, 29] with fitness 1.0363865546218487
Neighbor: [8, 22, 2, 0, 6, 10, 13, 11, 32, 17, 26, 25, 20, 15, 21, 12, 30, 27, 1, 7, 33, 34, 9, 3, 23, 28, 14, 18, 19, 31, 16, 4, 24, 5, 29] with fitness 1.0237264560996813
Neighbor: [14, 22, 2, 0, 6, 10, 13, 11, 16, 17, 26, 25, 20, 15, 21, 12, 30, 27, 1, 7, 33, 34, 9, 3, 23, 28, 8, 18, 19, 31, 32, 4, 24, 5, 29] with fitness 1.0542436974789915
Neighbor: [8, 11, 2, 0, 6, 10, 13, 22, 16, 17, 26, 25, 20, 15, 21, 12, 30, 27, 1, 7, 33, 34, 9, 3, 23, 28, 14, 18, 19, 31, 32, 4, 24, 5, 29] with fitness 1.0399579831932773
Neighbor: [11, 22, 2, 0, 6, 10, 13, 8, 16, 17, 26, 25, 20, 15, 21, 12, 30, 27, 1, 7, 33, 34, 9, 3, 23, 28, 14, 18, 19, 31, 32, 

In [67]:
visualize_musical_lineup(solution.repr, data.artists)

,Slot,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
0,Slot 1,14: Synthwave Saints|Rock|94,17: Nightfall Sonata|Classical|84,2: Velvet Pulse|Jazz|35,0: Midnight Echo|Rock|75,6: Aurora Skies|Pop|75
1,Slot 2,30: Turbo Vortex|Rock|53,13: Lunar Spectrum|Rock|99,21: Celestial Voyage|Electronic|95,26: Cloud Nine Collective|Pop|97,28: The Polyrhythm Syndicate|Jazz|66
2,Slot 3,12: Blue Horizon|Pop|51,25: Rhythm Alchemy|Jazz|94,5: Echo Chamber|Electronic|98,15: Golden Ember|Rock|61,22: Quantum Beat|Hip-Hop|96
3,Slot 4,4: The Silver Owls|Classical|85,10: The Wandering Notes|Jazz|84,27: Hypnotic Echoes|Rock|77,1: Solar Flare|Electronic|78,7: Static Mirage|Rock|94
4,Slot 5,33: Cosmic Frequency|Rock|53,34: Parallel Dimension|Electronic|58,9: Deep Resonance|Jazz|90,16: Mystic Rhythms|Classical|78,24: Shadow Cadence|Jazz|66
5,Slot 6,11: Phantom Groove|Hip-Hop|47,8: Crimson Harmony|Classical|20,18: Velvet Underground|Rock|72,31: The Jazz Nomads|Jazz|64,19: Astral Tide|Electronic|69
6,Slot 7,32: The Bassline Architects|Hip-Hop|61,20: The Sonic Drifters|Rock|88,3: Neon Reverie|Electronic|100,23: Electric Serpents|Electronic|99,29: Harmonic Dissonance|Classical|96


### Roulette Wheel